In [1]:
import cv2
import numpy as np
from mediapipe.tasks.python import vision
import mediapipe as mp
import time
from collections import defaultdict
import matplotlib.pyplot as plt
import os

2023-05-14 23:47:29.585979: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-14 23:47:39.364471: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-14 23:47:39.425954: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 23:48:03.064383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
objectDectector_model_filepath = "model/object_detector/efficientdet_lite0_uint8.tflite"
video_filepath = "dataset/video_cars.mp4"
embedder_model_filepath = "model/embedder/mobilenet_v3_small_075_224_embedder.tflite"

In [3]:
ObjecttDetectorOptions = mp.tasks.vision.ObjectDetectorOptions(
    base_options=mp.tasks.BaseOptions(model_asset_path=objectDectector_model_filepath),
    max_results=50,
    score_threshold=0.4,
    running_mode=mp.tasks.vision.RunningMode.VIDEO
)

    

In [4]:
cap = cv2.VideoCapture(video_filepath)
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("FPS: ",fps)


FPS:  60


In [6]:
# setting up video writer for testing the object detector
# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)
   
# Below VideoWriter object will create
# a frame of above defined The output 
# # is stored in 'filename.avi' file.
# videoWriter = cv2.VideoWriter('objDetect_video_cars_test.avi', 
#                          cv2.VideoWriter_fourcc(*'MJPG'),
#                          30, size)

In [6]:
frame_idx = 0
all_detections=defaultdict(list)
avg_processing_time = 0
with mp.tasks.vision.ObjectDetector.create_from_options(ObjecttDetectorOptions) as detector:
    while cap.isOpened():
        ret, img = cap.read()
        if ret == True:

            frame_idx+=1
            # Calculate the timestamp of the current frame
            frame_timestamp_ms = int(1000 * frame_idx / fps)
            # print(frame_timestamp_ms)
            time_start = time.perf_counter()
            # Convert the frame received from OpenCV to a MediaPipe’s Image object.
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)

            # Perform object detection on the video frame.
            detection_results = detector.detect_for_video(mp_image, frame_timestamp_ms)
            # print("time taken : ",time.perf_counter()-time_start)
            # print("Detections:",len(detection_results.detections))
            # print("--------------")
            for det_res in detection_results.detections:
                x = det_res.bounding_box.origin_x
                y = det_res.bounding_box.origin_y
                width = det_res.bounding_box.width
                height = det_res.bounding_box.height
                all_detections[frame_idx].append((x,y,x+width,y+height))
                # img = cv2.rectangle(img, (x,y),(x+width,y+height),color=(0,255,0),thickness=2)
            # cv2.imwrite("box_img.png",img)
            # videoWriter.write(img)
            # break
            time_processing = time.perf_counter()-time_start
            avg_processing_time += time_processing
        else:
            print("Not able to read the frame")
            break
cap.release()
# videoWriter.release()
print("time taken : ",avg_processing_time/frame_idx)
# print("Detections:",len(detection_results.detections))
print("--------------")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Not able to read the frame
time taken :  0.09556216746357879
--------------


In [8]:
detection_results.detections[0]

Detection(bounding_box=BoundingBox(origin_x=2269, origin_y=1173, width=497, height=388), categories=[Category(index=None, score=0.62890625, display_name=None, category_name='car')], keypoints=[])

In [7]:
# embeddings:
# Create options for Image Embedder
embedder_base_options = mp.tasks.BaseOptions(model_asset_path=embedder_model_filepath)
l2_normalize = True #@param {type:"boolean"}
quantize = True #@param {type:"boolean"}
options = vision.ImageEmbedderOptions(
    base_options=embedder_base_options, l2_normalize=l2_normalize, quantize=quantize,
    running_mode= mp.tasks.vision.RunningMode.VIDEO)


In [8]:
cap = cv2.VideoCapture(video_filepath)
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("FPS: ",fps)

# setting up video writer for testing the object detector
# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)
   
# Below VideoWriter object will create
# a frame of above defined The output 
# # is stored in 'filename.avi' file.
videoWriter = cv2.VideoWriter('objEmbed_video_cars_test.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)
output_image_dir = "images"
os.makedirs(output_image_dir,exist_ok=True)

FPS:  60


In [21]:
# Create Image Embedder
frame_idx = 0
prev_embeddings = [] # prev frame as 1, next frame as 2
next_embeddings = []
crop_idx = 0
frame_interval = 10 # after how many frames check similarity
avg_processing_time = 0
car_idx = 0 #car counts detected cars in the video, same car will be not counted more than 1
matching_thresh = 0.7

with vision.ImageEmbedder.create_from_options(options) as embedder:
   while cap.isOpened():
        ret, img = cap.read()
        if ret == True:

            frame_idx+=1
            if (frame_idx-1)%frame_interval==0:

                time_start = time.perf_counter()
                if len(prev_embeddings)==0:
                    embed_arr = []
                    car_idx_arr = []
                    # take first frame as the reference for starting tracking
                    for x1,y1,x2,y2 in all_detections[1]:
                        car_idx+=1
                        img_crop = img[y1:y2,x1:x2,:].astype(np.uint8)
                        crop_idx+=1
                        # Calculate the timestamp of the current frame
                        frame_timestamp_ms = int(1000 * crop_idx / fps)
                        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
                        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_crop)

                        # Perform object detection on the video frame.
                        embedding_result = embedder.embed_for_video(mp_image, frame_timestamp_ms)

                        embed_arr.append(embedding_result.embeddings[0])
                        car_idx_arr.append(car_idx)
                        # print(int(frame_idx//frame_interval),embedding_result)
                        prev_embeddings.append((img_crop,embedding_result,car_idx,(x1,y1,x2,y2)))
                    # print(embed_arr)
                    # break
                else:
                    for x1,y1,x2,y2 in all_detections[frame_idx]:
                        img_crop = img[y1:y2,x1:x2,:].astype(np.uint8)
                        crop_idx+=1
                        # Calculate the timestamp of the current frame
                        frame_timestamp_ms = int(1000 * crop_idx / fps)
                        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
                        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_crop)

                        # Perform object detection on the video frame.
                        embedding_result = embedder.embed_for_video(mp_image, frame_timestamp_ms)
                        em1 = embedding_result.embeddings[0]
                        best_match = 0
                        new_car_idx = None
                        for im_cr_prev, embed_prev, c_idx, coords in prev_embeddings:
                            em2 = embed_prev.embeddings[0]
                            similarity = vision.ImageEmbedder.cosine_similarity(
                                em1,
                                em2)

                            if similarity>matching_thresh and similarity>best_match:
                                best_match = similarity
                                new_car_idx = c_idx
                        if new_car_idx is None:
                            car_idx+=1
                            new_car_idx = car_idx
                        # print(int(frame_idx//frame_interval),embedding_result)
                        next_embeddings.append((img_crop,embedding_result,new_car_idx,(x1,y1,x2,y2)))
                    prev_embeddings = next_embeddings.copy()
                    next_embeddings = []
     
                for im_cr_prev, embed_prev, c_idx, coords in prev_embeddings:
                    x1 = coords[0]
                    y1 = coords[1]
                    x2 = coords[2]
                    y2 = coords[3]
                    img = cv2.putText(img, f'{c_idx}', org=(x1,y1),
                                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                        fontScale=3,
                                        color=(255,0,0),
                                        thickness=2,
                                        lineType=cv2.LINE_AA)
                    img = cv2.rectangle(img, (x1,y1),(x2,y2),color=(0,255,0),thickness=5)
                cv2.imwrite(os.path.join(output_image_dir,f"image_{frame_idx}.png"),img)
                videoWriter.write(img)
                try:    
                    avg_processing_time+=(time.perf_counter()-time_start)/((frame_idx-1)//frame_interval)
                except ZeroDivisionError as ZE:
                    avg_processing_time+=(time.perf_counter()-time_start)

        else:
            print("Not able to read the frame")
            break
cap.release()
videoWriter.release()
print("time taken : ",avg_processing_time)

Not able to read the frame
time taken :  6.232323939660664


# Adding Kalman filter

In [26]:
from filterpy.kalman import KalmanFilter
from tqdm import tqdm

In [28]:
cap = cv2.VideoCapture(video_filepath)
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("FPS: ",fps)

# setting up video writer for testing the object detector
# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)
   
# Below VideoWriter object will create
# a frame of above defined The output 
# # is stored in 'filename.avi' file.
videoWriter = cv2.VideoWriter('objEmbed_video_cars_test.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)
output_image_dir = "images"
os.makedirs(output_image_dir,exist_ok=True)

FPS:  60


In [51]:
# embeddings:
# Create options for Image Embedder
embedder_base_options = mp.tasks.BaseOptions(model_asset_path=embedder_model_filepath)
l2_normalize = True #@param {type:"boolean"}
quantize = True #@param {type:"boolean"}
emb_options = vision.ImageEmbedderOptions(
    base_options=embedder_base_options, l2_normalize=l2_normalize, quantize=quantize,
    running_mode= mp.tasks.vision.RunningMode.VIDEO)

ObjecttDetectorOptions = mp.tasks.vision.ObjectDetectorOptions(
    base_options=mp.tasks.BaseOptions(model_asset_path=objectDectector_model_filepath),
    max_results=30,
    score_threshold=0.4,
    running_mode=mp.tasks.vision.RunningMode.VIDEO
)

In [52]:

class Track():
    def __init__(self, track_idx, bbox, features, frame_limit =10, input_st=7, output_st=4):
        # input_st = 7
        # output_st = 4
        
        self.warming = True
        self.warming_count = 5 # that many frames needed to predict accuratly for the pose
        self.set_kf(input_st, output_st)
        self.kf.x[:4] = self.convert_bbox_to_z(bbox)

        self.track_idx = track_idx
        self.obj_not_seen = 0 # object not seen for certain frames count
        self.frame_limit = frame_limit
        self.features = features

        self.time_since_update = 0

    def notFound(self):
        if self.time_since_update>self.frame_limit:
            return True
        return False

    def set_kf(self, input_st, output_st):
        self.kf = KalmanFilter(dim_x=input_st, dim_z=output_st)

        # A
        self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])

        # H
        self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01

    def convert_bbox_to_z(self,bbox):
        """
        Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
        [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
        the aspect ratio
        """
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        x = bbox[0] + w/2.
        y = bbox[1] + h/2.
        s = w * h    #scale is just area
        r = w / float(h)
        return np.array([x, y, s, r]).reshape((4, 1))

    def convert_x_to_bbox(self,x):
        w = np.sqrt(x[2] * x[3])
        h = x[2] / w
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
    
    def update(self,bbox, features):
        """
        Updates the state vector with observed bbox.
        """
        self.time_since_update = 0
        history = []

        # update features
        prev_feature = self.features.embedding
        new_feature = features.embedding
        self.features.embedding = (prev_feature+new_feature)/2

        self.kf.update(self.convert_bbox_to_z(bbox))
        if self.warming:
            self.warming_count-=1
            if self.warming_count<=0:
                self.warming=False

    def predict(self):
        """
        Advances the state vector and returns the predicted bounding box estimate.
        """
        if((self.kf.x[6]+self.kf.x[2])<=0):
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.time_since_update += 1
        # history.append()
        return self.convert_x_to_bbox(self.kf.x)
        # return history[-1]


In [53]:
def IOU(bbox1,bbox2):
	
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(bbox1[0], bbox2[0])
	yA = max(bbox1[1], bbox2[1])
	xB = min(bbox1[2], bbox2[2])
	yB = min(bbox1[3], bbox2[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	bbox1Area = (bbox1[2] - bbox1[0] + 1) * (bbox1[3] - bbox1[1] + 1)
	bbox2Area = (bbox2[2] - bbox2[0] + 1) * (bbox2[3] - bbox2[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(bbox1Area + bbox2Area - interArea)
	# return the intersection over union value
	return iou

In [54]:
frame_idx = 0
# Embedding with tracking
# Create Image Embedder
crop_idx = 0

all_detections=defaultdict(list)
avg_processing_time = 0
# store_images = {}

with vision.ImageEmbedder.create_from_options(emb_options) as embedder:
    with mp.tasks.vision.ObjectDetector.create_from_options(ObjecttDetectorOptions) as detector:
        while cap.isOpened():
            ret, img = cap.read()
            if ret == True:

                frame_idx+=1
                # Calculate the timestamp of the current frame
                frame_timestamp_ms = int(1000 * frame_idx / fps)
                # print(frame_timestamp_ms)
                time_start = time.perf_counter()
                # Convert the frame received from OpenCV to a MediaPipe’s Image object.
                mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)

                # Perform object detection on the video frame.
                detection_results = detector.detect_for_video(mp_image, frame_timestamp_ms)
                # print("time taken : ",time.perf_counter()-time_start)
                # print("Detections:",len(detection_results.detections))
                # print("--------------")
                # crop_idx=0
                for det_res in detection_results.detections:
                    x = det_res.bounding_box.origin_x
                    y = det_res.bounding_box.origin_y
                    width = det_res.bounding_box.width
                    height = det_res.bounding_box.height
                    
                    x1,y1,x2,y2 = (x,y,x+width,y+height)
                    img_crop = img[y1:y2,x1:x2,:].astype(np.uint8)
                    crop_idx+=1
                    # Calculate the timestamp of the current frame
                    frame_timestamp_ms = int(1000 * crop_idx / fps)
                    # Convert the frame received from OpenCV to a MediaPipe’s Image object.
                    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_crop)

                    # Perform object detection on the video frame.
                    embedding_result = embedder.embed_for_video(mp_image, frame_timestamp_ms)

                    # embed_arr.append(embedding_result.embeddings[0])
                    features = embedding_result.embeddings[0]
                    all_detections[frame_idx].append(((x1,y1,x2,y2), features))
                    # img = cv2.rectangle(img, (x,y),(x+width,y+height),color=(0,255,0),thickness=2)
                # store_images[frame_idx] = store_images.get(frame_idx, img)
                # cv2.imwrite("box_img.png",img)
                # videoWriter.write(img)
                # break
                time_processing = time.perf_counter()-time_start
                avg_processing_time += time_processing
            else:
                print("Not able to read the frame")
                break

            if frame_idx>200:
                break
cap.release()
# videoWriter.release()
print("time taken : ",avg_processing_time/frame_idx)
# print("Detections:",len(detection_results.detections))
print("--------------")

time taken :  0.13375994990048495
--------------


In [56]:
cap = cv2.VideoCapture(video_filepath)
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("FPS: ",fps)

# setting up video writer for testing the object detector
# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
   
# Below VideoWriter object will create
# a frame of above defined The output 
# # is stored in 'filename.avi' file.
videoWriter = cv2.VideoWriter('objEmbed_video_cars_track.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)
output_image_dir = "images"
os.makedirs(output_image_dir,exist_ok=True)

FPS:  60


## TODO
* Add NMS

In [57]:

frame_idx = 0
prev_embeddings = [] # prev frame as 1, next frame as 2
next_embeddings = []

frame_interval = 10 # after how many frames check similarity
avg_processing_time = 0
car_idx = 0 #car counts detected cars in the video, same car will be not counted more than 1
matching_thresh = 0.7

obj_track = {}
iou_thresh = 0.7
prev_embeddings = []
# with vision.ImageEmbedder.create_from_options(options) as embedder:
while cap.isOpened():
    ret, img = cap.read()
    if ret == True:
        frame_idx+=1
        print("processing frame : ",frame_idx)
        values  = all_detections[frame_idx]
        time_start = time.perf_counter()
        # img = store_images[frame_idx]
        if len(prev_embeddings)==0:
            embed_arr = []
            car_idx_arr = []
            # take first frame as the reference for starting tracking
            for bbox, features in values:
                car_idx+=1

                # embed_arr.append(embedding_result.embeddings[0])
                car_idx_arr.append(car_idx)
                obj_track[car_idx] = obj_track.get(car_idx, Track(car_idx,bbox=bbox, features=features))

                # print(int(frame_idx//frame_interval),embedding_result)
                prev_embeddings.append((features,car_idx,bbox))
            # print(obj_track)
            # break
        else:
            for bbox, features in values:
                em1 = features
                best_match = 0
                new_car_idx = None
                pe = prev_embeddings.copy()
                idx_remove = None
                best_iou = 0
                best_iou_idx = 0
                for idx, (embed_prev, c_idx, coords) in enumerate(prev_embeddings):
                    em2 = embed_prev
                    similarity = vision.ImageEmbedder.cosine_similarity(
                        em1,
                        em2)
                    
                    iou = IOU(bbox, coords)
                    if iou>iou_thresh and iou>best_iou:
                        best_iou = iou
                        best_iou_idx = idx

                    if similarity>matching_thresh and similarity>best_match:
                        best_match = similarity
                        new_car_idx = c_idx
                        idx_remove = idx
                if new_car_idx is None:
                    flag= True
                    if best_iou_idx!=0:
                        bbox_p = obj_track[best_iou_idx].predict().squeeze().tolist()
                        iou = IOU(bbox_p,bbox)
                        print(iou)
                        if iou>0.95:
                            print("Got data but embeddings are not matching")
                            # obj_track[best_iou_idx].predict()
                            obj_track[best_iou_idx].update(bbox=bbox,features=features)
                            flag= False
                            new_car_idx = best_iou_idx
                            
                    if flag:
                        car_idx+=1
                        new_car_idx = car_idx
                        obj_track[car_idx] = obj_track.get(car_idx, Track(car_idx, bbox=bbox, features=features))
                else:
                    prev_embeddings.pop(idx_remove)
                    x1,y1,x2,y2 = bbox
                    _ = obj_track[new_car_idx].predict()
                    # print(features)
                    # print(features.embedding)
                    obj_track[new_car_idx].update(bbox=[x1,y1,x2,y2],features=features)
                # print(int(frame_idx//frame_interval),embedding_result)
                next_embeddings.append((features,new_car_idx,bbox))
            # reduce the count of predict for the cars not found
            for _,c_id,_ in prev_embeddings:
                ret = obj_track[c_id].notFound()
                if ret:
                    # object not found for certain frames deleting the object
                    del obj_track[c_id]
            prev_embeddings = next_embeddings.copy()
            next_embeddings = []

        for embed_prev, c_idx, coords in prev_embeddings:
            x1 = coords[0]
            y1 = coords[1]
            x2 = coords[2]
            y2 = coords[3]
            img = cv2.rectangle(img, (x1,y1),(x1+3,y1+3),color=(255,0,0),thickness=-1)
            img = cv2.putText(img, f'{c_idx}', org=(x1,y1),
                                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                fontScale=3,
                                color=(255,255,255),
                                thickness=2,
                                lineType=cv2.LINE_AA)
            img = cv2.rectangle(img, (x1,y1),(x2,y2),color=(0,255,0),thickness=5)
        cv2.imwrite(os.path.join(output_image_dir,f"image_{frame_idx}.png"),img)
        videoWriter.write(img)
        try:    
            avg_processing_time+=(time.perf_counter()-time_start)/((frame_idx-1)//frame_interval)
        except ZeroDivisionError as ZE:
            avg_processing_time+=(time.perf_counter()-time_start)

    # else:
    #     print("Not able to read the frame")
    #     break
cap.release()
videoWriter.release()
print("time taken : ",avg_processing_time)

processing frame :  1
processing frame :  2
processing frame :  3
processing frame :  4
processing frame :  5
processing frame :  6
processing frame :  7
processing frame :  8
processing frame :  9
processing frame :  10
processing frame :  11
processing frame :  12
processing frame :  13
processing frame :  14
processing frame :  15
processing frame :  16
processing frame :  17
processing frame :  18
processing frame :  19
processing frame :  20
processing frame :  21
processing frame :  22
processing frame :  23
processing frame :  24
processing frame :  25
processing frame :  26
processing frame :  27
processing frame :  28
processing frame :  29
processing frame :  30
processing frame :  31
processing frame :  32
processing frame :  33
processing frame :  34
processing frame :  35
processing frame :  36
processing frame :  37
processing frame :  38
processing frame :  39
processing frame :  40
processing frame :  41
processing frame :  42
processing frame :  43
processing frame :  

KeyboardInterrupt: 

In [46]:
obj_track[174].predict()

KeyError: 174

In [47]:
obj_track

{1: <__main__.Track at 0x7f50d655fca0>,
 2: <__main__.Track at 0x7f50ab358310>,
 3: <__main__.Track at 0x7f50c866ef40>,
 4: <__main__.Track at 0x7f50c866ef10>,
 5: <__main__.Track at 0x7f50c866eeb0>,
 6: <__main__.Track at 0x7f50c864e040>,
 7: <__main__.Track at 0x7f50c864e100>,
 8: <__main__.Track at 0x7f50c8686370>,
 9: <__main__.Track at 0x7f50cde91b50>,
 10: <__main__.Track at 0x7f50cde91b80>,
 11: <__main__.Track at 0x7f50cde860d0>,
 12: <__main__.Track at 0x7f50cde91f40>,
 13: <__main__.Track at 0x7f50cde91e20>,
 14: <__main__.Track at 0x7f50cde91bb0>,
 15: <__main__.Track at 0x7f50cde91fa0>,
 16: <__main__.Track at 0x7f50cde91f70>,
 17: <__main__.Track at 0x7f50cde91c40>,
 18: <__main__.Track at 0x7f50cde91cd0>,
 19: <__main__.Track at 0x7f50cde91b20>,
 20: <__main__.Track at 0x7f50cde91be0>,
 21: <__main__.Track at 0x7f50cde91d90>,
 22: <__main__.Track at 0x7f50cde91970>,
 23: <__main__.Track at 0x7f50cde86070>,
 24: <__main__.Track at 0x7f50cde91a90>,
 25: <__main__.Track at 0

In [50]:
obj_track[103].predict().squeeze().tolist()

[2338.0, 1684.0, 3045.0, 2170.0]